### Notebook for hyperparameter tuning (using optuna package)
#### (all adapted to dog dataset)

##### optuna installation and imports

In [ ]:
!pip install optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
import optuna
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset
from tqdm import tqdm
import tqdm

##### Creating PyTorch dataloaders for loading the data into the models later on

In [ ]:
low_res_folder = "/kaggle/working/dog_low_res"
high_res_folder = "/kaggle/input/animal-faces/afhq/train/dog"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



### --------------------------------------------------------------------------------- ###
#if you use the Human Data --->
"""# # === train_data augmentation ===

# # color jitter augmentation for training
train_color_jitter = ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)

# # augmentation factor
augmentation_factor = 1

# # augmented datasets with random color jitter
augmented_datasets = []
for _ in tqdm(range(augmentation_factor)):
    augmented_dataset = []
    for image, target in train_dataset:
        augmented_dataset.append((train_color_jitter(image), train_color_jitter(target)))
    augmented_datasets.append(augmented_dataset)

# # combine original and augmented datasets
combined_datasets = [train_dataset] + augmented_datasets
train_dataset = ConcatDataset(combined_datasets)"""
### --------------------------------------------------------------------------------- ###



# === final data loaders ===

# Data loaders for train and val sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")





#### tuning hyperparameters for all models
##### following below each model will have two code cells, one for defining the model itself and the other one for tuning its hyperparameters using optuna

#### SRCNN

In [ ]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

In [ ]:
num_epochs = 10
def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    model = SRCNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in tqdm(range(num_epochs), desc="Training"):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=5)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()
df.to_csv('study_results_SRCNN.csv', index=False)


### FSRCNN

In [ ]:
# Load the trained model
class FSRCNN(nn.Module):
    def __init__(self, d=116, s=15, m=3):
        super(FSRCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)

        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)

        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])

        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        
        # Deconvolution for upscaling to desired size
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=4, padding=4, output_padding=3)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [ ]:
num_epochs = 10
def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    d = trial.suggest_int('d', 32, 128)
    s = trial.suggest_int('s', 5, 20)
    m = trial.suggest_int('m', 2, 8)
    model = FSRCNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=25)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()
df.to_csv('/study_results_FSRCNN.csv', index=False)

### ESPCN

In [ ]:
class ESPCN(nn.Module):
    def __init__(self, upscale_factor=4, num_channels=3):
        super(ESPCN, self).__init__()
        self.upscale_factor = upscale_factor
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        
        # Subpixel convolution with pixle shuffle for upscaling to desired size
        self.conv2 = nn.Conv2d(64, num_channels * upscale_factor ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pixel_shuffle(self.conv2(x))
        x = self.relu2(x)
        return x

In [ ]:
num_epochs = 20
def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 5e-4)
    model = ESPCN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in tqdm(range(num_epochs), desc="Training"):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=2)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()
df.to_csv('study_results_ESPCN.csv', index=False)